###**LTC or pressure datalogger correction for barometric pressure fluctuations**

In [ ]:
#Connect to Drive
from google.colab import drive
drive.mount("/content/drive/")
import pandas as pd
import os

Specify file names for LTC and barometric files. Path name should be the same- make sure working files are in the preprocess_files folder.

In [ ]:
#Rename the datfile and barofile to the filenames you want to use
path='/content/drive/My Drive/CZN_hydro/Water/preprocess_files/'
datfile='BN_A_20190314_LT.csv'
barofile='BN_20190314_Baro.csv'

For **Onset HOBO loggers**, run the 3 following code blocks.
Skip this part if you're using another brand.

In [ ]:
#1
#Open data logger file, skip first row, only first 4 columns - *You may need to add another column if this is a LTC vs a LT*
#Add "encoding='gbk'" if degrees symbol doesn't show up
df=pd.read_csv(path+datfile,encoding='gbk',skiprows=1,usecols = [i for i in range(4)])
df.rename(columns = {list(df)[1]:'DateTime',list(df)[2]:'LOGGER_P_kPa',
                     list(df)[3]:'LOGGER_Temp_C'}, inplace = True)
df['DateTime']=pd.to_datetime(df['DateTime'])
df=df.set_index('DateTime')
df=df.drop(['#'],axis=1)
#Drop any rows at the end with no value
s=str(df['LOGGER_P_kPa'].last_valid_index())
df=df[df.index<=s]

#Bring in barometric data
barodf=pd.read_csv(path+barofile,skiprows=1,usecols = [i for i in range(4)])
barodf.rename(columns = {list(barodf)[1]:'DateTime',list(barodf)[2]:'BARO_P_kPa',
                         list(barodf)[3]:'BARO_Temp_C'}, inplace = True) 
barodf['DateTime']=pd.to_datetime(barodf['DateTime'])
barodf=barodf.set_index('DateTime')
barodf=barodf.drop(['#'],axis=1)

In [ ]:
#2
#Convert baro pressure to meters water
df.rename(columns = {'BARO_P_kPa':'BARO_mWater'}, inplace=True)
df['BARO_mWater']=(df['BARO_mWater']*KPA_PSI*PSI_FT*FT_M)
BP1=df['BARO_mWater'][0]

In [ ]:
#3
#Get the temperature-dependent density compensation correction values
te=df['LOGGER_Temp_C']
p_lbft3=(((999.83952+16.945176*te)-(7.9870401e-03*(te**2))-(46.170461e-06*(te**3))+
      (105.56302e-09*(te**4)) - (280.54253e-12*(te**5))) / (1 + 16.879850e-03*te))*0.0624279606
#Conversion constants
PSI_PSF=144
KPA_PSI=0.1450377
FT_M=0.3048
PSI_FT=2.0366

#Convert logger pressure to meters water with density compensation
df['LOGGER_mWater_corr']=((KPA_PSI*PSI_PSF*df['LOGGER_P_kPa']/p_lbft3)*FT_M)+(BP1-df['BARO_mWater'])

df.head()

For **Solinst LTC loggers**, run the 3 following code blocks.
Skip this part if you're using another brand.

In [ ]:
#1
#Import custom function - available for download on Github as find_first_row.py
import sys
sys.path.append('/content/drive/My Drive/CZN_hydro/Water/process_scripts')
import find_first_row
#Solinst typically have some logger metadata at the top of the file; this is to skip over that
ffr=find_first_row.first_row('/content/drive/My Drive/CZN_hydro/Water/preprocess_files/'+file)
ffrbaro=find_first_row.first_row('/content/drive/My Drive/CZN_hydro/Water/preprocess_files/'+barofile)
#print(ffr)

#Open data logger file
df=pd.read_csv(path+datfile,skiprows=ffr)
DateTime=df['Date']+' '+df['Time']
df.insert(0,'DateTime',DateTime)
df['DateTime']=pd.to_datetime(df['DateTime']).dt.round('15min').dt.strftime('%m/%d/%Y %H:%M:%S')
df.rename(columns={'LEVEL':'LOGGER_mWater','TEMPERATURE':'LOGGER_Temp_C','CONDUCTIVITY':'LOGGER_EC'})
df=df.set_index('DateTime')

#Bring in barometric data
barodf=pd.read_csv(path+barofile,skiprows=ffrbaro)
DateTime=df['Date']+' '+df['Time']
df.insert(0,'DateTime',DateTime)
df['DateTime']=pd.to_datetime(df['DateTime']).dt.round('15min').dt.strftime('%m/%d/%Y %H:%M:%S')
df.rename(columns={'LEVEL':'BARO_P_kPa','TEMPERATURE':'BARO_Temp_C'})
barodf['DateTime']=pd.to_datetime(barodf['DateTime'])
barodf=barodf.set_index('DateTime')

In [ ]:
#2
#Merge two dataframes, keeps ALL LTC records and interpolates any missing baro P values
df=df.merge(barodf['BARO_P_kPa'],how='left',left_index=True,right_index=True)
df['BARO_P_kPa']=df['BARO_P_kPa'].interpolate(axis=0)

In [ ]:
#3
#Convert baro pressure to meters water
df.rename(columns = {'BARO_P_kPa':'BARO_mWater'}, inplace=True)
df['BARO_mWater']=(df['BARO_mWater']*KPA_PSI*PSI_FT*FT_M)
BP1=df['BARO_mWater'][0]

#Correct for baro pressure
df['LOGGER_mWater_corr']=df['LOGGER_mWater']+(BP1-df['BARO_mWater'])

df.head()

Run the following for ALL logger brands.

In [ ]:
#Save a copy to the folder with a suffix and delete old copy
df.to_csv(path+datfile.split('.')[0]+'_baroCorrect.csv')
os.remove(path+datfile)